In [15]:
import json

import psycopg2
import gzip
import pandas as pd

In [16]:
with open('secret.json') as fp:
    settings = json.load(fp)

conn = psycopg2.connect(dbname="imdb", user=settings['username'], password=settings['password'], host = settings['host'], port = settings['port'])

cur = conn.cursor()

cur.execute('DROP SCHEMA IF EXISTS public CASCADE')
cur.execute('CREATE SCHEMA public')

with open('ImdbCreateSchema.sql') as fp:
    queries = fp.read()
cur.execute(queries)



In [18]:
with gzip.open('dataset/title.basics.tsv.gz') as fp:
    data = pd.read_csv(fp, sep='\t')
data

/tmp/ipykernel_3859/1010911268.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(fp, sep='\t')


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
9292079,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,\N,\N,"Action,Drama,Family"
9292080,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,"Action,Drama,Family"
9292081,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family"
9292082,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short


In [25]:
data[data['genres'].map(type)!=str]['genres']

1097688    NaN
1509021    NaN
1898525    NaN
2009609    NaN
2164388    NaN
2309864    NaN
3001220    NaN
6612186    NaN
9248683    NaN
9288535    NaN
Name: genres, dtype: object

In [26]:

data['isAdult'] = data['isAdult'].astype(bool)       
data['startYear'] = data.startYear.replace(r'\N', None)
data['runtimeMinutes'] = data.runtimeMinutes.replace(r'\N', None)
# print(data[1:3])
data.replace("NONE", None)
data['endYear'] = data.endYear.replace(r'\N', None)  
# print(data)

data['genres'] = data['genres'].map(lambda s: s.split(',') if type(s)==str else [])


: 

: 

In [ ]:

data[data['genres'].map(lambda x: type(x))!=str]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
60162,tt0061366,movie,Around the World,Around the World,False,None,None,None,None
65731,tt0067098,tvEpisode,Willi Forst,Willi Forst,False,None,None,None,None
78421,tt0080155,movie,The Wonderful Years,Die wunderbaren Jahre,False,None,None,None,None
83777,tt0085677,tvEpisode,High Country,High Country,False,None,None,None,None
90900,tt0092975,tvEpisode,Erste Liebe und Heiratssachen,Erste Liebe und Heiratssachen,False,None,None,None,None
...,...,...,...,...,...,...,...,...,...
9291971,tt9916616,short,Terror,Terror,False,None,None,None,None
9291973,tt9916620,movie,The Copeland Case,The Copeland Case,False,None,None,None,None
9291987,tt9916652,short,Untitled well- being Documentary,Untitled well- being Documentary,False,None,None,None,None
9291993,tt9916664,short,Untitled Land Army Girls Documentary,Untitled Land Army Girls Documentary,False,None,None,None,None


In [ ]:

# print(data[1:4])
#print(data[1:8])
#print(data['tconst'].c)
cursor = conn.cursor()
#cursor = conn.cursor('cursor_unique_name', cursor_factory=psycopg2.extras.DictCursor)


cursor.executemany('INSERT INTO titlebasics VALUES (%s , %s, %s, %s, %s, %s, %s, %s, %s)', (data.values.tolist()))

cursor.commit()
    #cursor.execute('INSERT INTO titlebasics(tconst, titleType,primaryTitle, originalTitle, isAdult, startYear,endYear, genres, runTimeMinutes ) values (%s , %s, %s, %s, %s, %s, %s, %s, %s)', (row[0], row[1], row[2],row[3], row[4], row[5],row[6], row[8], row[7]) )

    #cursor.execute('INSERT INTO titlebasics(tconst, titleType,primaryTitle, originalTitle, isAdult, startYear,endYear, genres, runTimeMinutes ) values (%s , %s, %s, %s, %s, %s, %s, %s, %s)', (row["tconst"], row["titleType"], row["primaryTitle"],row["originalTitle"], row["isAdult"], row["startYear"],row["endYear"], row["genres"], row["runtimeMinutes"]) )
# print(row)
# for row in data:
#     row.replace("[", "(", 1)
#     row.replace("]", ")", len(row))
#     cursor.execute('INSERT INTO titleakas' + row )
# for row in data:
#     row.replace("[", "(", 1)
#     row.replace("]", ")", len(row))
#     cursor.execute('INSERT INTO title' + row )
#cursor.execute('SELECT * FROM titlebasics LIMIT 50')


InvalidTextRepresentation: malformed array literal: "1894"
LINE 1: ...mencita', 'Carmencita', false, '1894', NULL, '1894', '1894')
                                                                ^
DETAIL:  Array value must start with "{" or dimension information.
